In [ ]:
import numpy as np
from random import *
import tifffile as tiff
import imageio as io
import cv2
import os
from matplotlib import pyplot as plt
from sklearn.feature_extraction import image
from sklearn.preprocessing import MinMaxScaler as mm
import h5py

In [ ]:
i=0
for ima in os.listdir('/home/salijona/Documents/UT/NeuralNetworksProject/Noisy_Images/Noisy_Images/'):
    i=i+1
print(i)

In [ ]:
#original_full=np.ndarray(shape=(840,1080,1080))
#noisy_full=np.ndarray(shape=(840,1080,1080))

scaler=mm(feature_range=(0, 1))
img_name=[]
i=0
for ima in os.listdir('/home/salijona/Documents/UT/NeuralNetworksProject/TIFF/TIFF_Original/Images'):
    img_in = np.array(cv2.imread('/home/salijona/Documents/UT/NeuralNetworksProject/TIFF/TIFF_Original/Images/'+ima,cv2.IMREAD_ANYDEPTH), dtype=np.uint16)
    img_in=img_in.clip(0)
   
    plt.figure(figsize=(20,10))

    plt.imshow(img_in, cmap="gray")
    plt.show()
   # scaler.fit(img_in)
    #t = scaler.transform(img_in)
    #img_in =t.reshape(img_in.shape)

    rmin=np.min(img_in)
    rmax=np.max(img_in)
    img_in=(img_in-rmin)/(rmax-rmin)
    #img_in = int(255 * (img_in - rmin) / (rmax - rmin) )
    img_in=img_in.clip(0)
    img_in=np.clip(img_in,a_min=0,a_max=1)
    plt.figure(figsize=(20,10))

    plt.imshow(img_in, cmap="gray")
    plt.show()
    original_full[i]=img_in
    print(np.min(img_in))
    print(np.max(img_in))

    
    img_in = np.array(cv2.imread('/home/salijona/Documents/UT/NeuralNetworksProject/Noisy_Images/Noisy_Images/'+ima,cv2.IMREAD_ANYDEPTH), dtype=np.int16)
    img_in=img_in.clip(0)

    rmin=np.min(img_in)
    rmax=np.max(img_in)
    img_in=(img_in-rmin)/(rmax-rmin)
    #img_in = int(255 * (img_in - rmin) / (rmax - rmin) )
    img_in=img_in.clip(0)
    img_in=np.clip(img_in,a_min=0,a_max=1)
    plt.figure(figsize=(20,10))

    plt.imshow(img_in, cmap="gray")
    plt.show()

    noisy_full[i]=img_in

    
    
    
    i=i+1

with h5py.File('original_full.h5', 'w') as hf:
    hf.create_dataset("original_full",  data=original_full)

with h5py.File('noisy_full.h5', 'w') as hf:
    hf.create_dataset("noisy_full",  data=noisy_full)

#np.save('original_full',original_full)
#np.save('noisy_full',noisy_full)

del original_full
del noisy_full

In [ ]:
f = h5py.File('noisy_full.h5', 'r')
all_data = np.array(f.get('noisy_full'))[:, :, :]
f.close()



n_inst = all_data.shape[0] #total number of images
np.random.seed(22)
indices = np.random.permutation(n_inst) #
n_train = int(0.8 * n_inst)
training_idx, test_idx = indices[:n_train], indices[n_train:]

data, test_data  = all_data[training_idx,:], all_data[test_idx,:]


with h5py.File('data.h5', 'w') as hf:
    hf.create_dataset("data",  data=data)    
    
with h5py.File('test_data.h5', 'w') as hf:
    hf.create_dataset("test_data",  data=test_data)  

    
    
del all_data
del data
del test_data

 

In [ ]:

f = h5py.File('original_full.h5', 'r')
all_label = np.array(f.get('original_full'))[:, :, :]
f.close()

n_inst = all_label.shape[0] #total number of images
np.random.seed(22)
indices = np.random.permutation(n_inst) #
n_train = int(0.8 * n_inst)
training_idx, test_idx = indices[:n_train], indices[n_train:]


    
label, test_label = all_label[training_idx,:], all_label[test_idx,:]

  
    
with h5py.File('label.h5', 'w') as hf:
    hf.create_dataset("label",  data=label)    

    
with h5py.File('test_label.h5', 'w') as hf:
    hf.create_dataset("test_label",  data=test_label)    
del all_label   
del label
del test_label

In [ ]:

f = h5py.File('data.h5', 'r')
data = np.array(f.get('data'))[:, :, :]
f.close()

data_patches=[]
for img in data: 
    patches=image.extract_patches_2d(img, patch_size=(64,64), max_patches=16,   random_state=0)
    data_patches.append(patches)
    
data_patches=np.vstack(data_patches)
print(data_patches.shape)
with h5py.File('data_patches.h5', 'w') as hf:
    hf.create_dataset("data_patches",  data=data_patches)    

    
del data_patches

f = h5py.File('test_data.h5', 'r')
test_data = np.array(f.get('test_data'))[:, :, :]
f.close()

test_data_patches=[]
for img in test_data: 
    patches=image.extract_patches_2d(img, patch_size=(64,64), max_patches=16,   random_state=0)
    test_data_patches.append(patches)

test_data_patches=np.vstack(test_data_patches)
print(test_data_patches.shape)
with h5py.File('test_data_patches.h5', 'w') as hf:
    hf.create_dataset("test_data_patches",  data=test_data_patches)  

del data
del test_data
del data_patches
del test_data_patches

In [ ]:
f = h5py.File('label.h5', 'r')
label = np.array(f.get('label'))[:, :, :]
f.close()

label_patches=[]
for img in label: 
    patches=image.extract_patches_2d(img, patch_size=(64,64), max_patches=16,   random_state=0)
    label_patches.append(patches)
    
label_patches=np.vstack(label_patches)
print(label_patches.shape)
with h5py.File('label_patches.h5', 'w') as hf:
    hf.create_dataset("label_patches",  data=label_patches)    

    
del label_patches

f = h5py.File('test_label.h5', 'r')
test_label = np.array(f.get('test_label'))[:, :, :]
f.close()

test_label_patches=[]
for img in test_label: 
    patches=image.extract_patches_2d(img, patch_size=(64,64), max_patches=16,   random_state=0)
    test_label_patches.append(patches)

test_label_patches=np.vstack(test_label_patches)
print(test_label_patches.shape)
with h5py.File('test_label_patches.h5', 'w') as hf:
    hf.create_dataset("test_label_patches",  data=test_label_patches)  
del label
del test_label
del label_patches
del test_label_patches